In [26]:
# import laibrary
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Bidirectional, Embedding


import pandas as pd
import numpy as np
import nltk
import re
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

In [27]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/dev/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [28]:
## read dataset
df = pd.read_csv('train.csv')

In [29]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [30]:
df.shape

(20800, 5)

In [31]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [32]:
df = df.dropna()

In [33]:
df.shape

(18285, 5)

In [34]:
# split independent feature and dependent feature
X = df.drop('label', axis=1)
y = df['label']

In [36]:
messages = X.copy()
set_stopwords = stopwords.words('english') 

In [37]:
messages.reset_index(inplace=True)

In [38]:
## Data preprocessing
ps = PorterStemmer()
corpus = []
for i in range(len(messages)):
    review = re.sub('[^a-zA-z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if word not in set_stopwords]
    review = ' '.join(review)
    corpus.append(review)

In [39]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic [photos]',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hi

In [41]:
## one hot embeding
voc_size = 20
one_hot_repr = [one_hot(word, voc_size) for word in corpus]
one_hot_repr

[[3, 6, 16, 12, 19, 6, 2, 3, 1, 17],
 [5, 14, 1, 11, 2, 9, 10],
 [15, 9, 5, 16],
 [5, 8, 1, 1, 13, 11],
 [11, 2, 1, 10, 17, 10, 2, 7, 12, 12],
 [4, 3, 13, 15, 4, 2, 2, 8, 8, 1, 17, 2, 7, 13, 10],
 [17, 11, 17, 2, 9, 16, 3, 10, 11, 3, 15],
 [7, 8, 15, 17, 5, 13, 2, 12, 11, 3, 15],
 [3, 9, 5, 11, 8, 18, 7, 17, 2, 8],
 [5, 14, 1, 13, 15, 9, 11, 10],
 [16, 5, 1, 19, 18, 5, 8, 4, 10, 17, 15],
 [1, 1, 19, 18, 2, 5],
 [2, 14, 19, 10, 7, 14, 12, 8, 6],
 [12, 17, 19, 11, 18, 8, 15, 11, 3, 15],
 [9, 3, 5, 6, 3, 11, 3, 15],
 [19, 1, 11, 5, 19, 2, 15, 14, 11, 15],
 [9, 11, 14],
 [12, 4, 6, 16, 2, 16, 8, 10],
 [2, 10, 1, 5, 6, 14, 14, 9, 10],
 [8, 11, 2, 1, 2, 10],
 [3, 4, 3, 19, 4, 14, 1, 14, 1, 11, 3, 15],
 [12, 8, 9, 7, 10, 5, 10],
 [7, 17, 17, 16, 15, 7, 9, 19, 12, 6, 7, 10],
 [9, 14, 14, 3, 17, 16, 1, 6],
 [3, 13, 5, 11, 1, 2, 5, 8, 13, 11, 3, 15],
 [14, 1, 19, 8, 11, 3, 15],
 [12, 4, 6, 3, 14, 14, 11, 10],
 [19, 1, 18, 3, 12, 11, 3, 15],
 [15, 5, 15, 10, 7, 15, 19, 11, 15, 11, 3, 15],
 [18, 8

In [42]:
## embedding representation
sent_sength = 20
embedded_docs = pad_sequences(one_hot_repr,padding='pre' , maxlen = sent_sength)
embedded_docs

array([[ 0,  0,  0, ...,  3,  1, 17],
       [ 0,  0,  0, ...,  2,  9, 10],
       [ 0,  0,  0, ...,  9,  5, 16],
       ...,
       [ 0,  0,  0, ..., 11,  3, 15],
       [ 0,  0,  0, ...,  3,  4, 16],
       [ 0,  0,  0, ..., 16,  2,  9]], dtype=int32)

In [44]:
## creating model
embedded_feature_length = 40
model = Sequential()
model.add(Embedding(voc_size, embedded_feature_length, input_length=sent_sength))
model.add(Bidirectional(LSTM(100)))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer = 'adam', metrics=['accuracy'])

In [45]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 20, 40)            800       
                                                                 
 bidirectional_1 (Bidirecti  (None, 200)               112800    
 onal)                                                           
                                                                 
 dense (Dense)               (None, 1)                 201       
                                                                 
Total params: 113801 (444.54 KB)
Trainable params: 113801 (444.54 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [46]:
X_final = np.array(embedded_docs)
y_final = np.array(y)

In [47]:
## train test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.25, random_state=42)

In [48]:
## Finally Trainning
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=15, batch_size = 64)

Epoch 1/15
215/215 [==============================] - 11s 34ms/step - loss: 0.4148 - accuracy: 0.8133 - val_loss: 0.3501 - val_accuracy: 0.8521
Epoch 2/15
215/215 [==============================] - 6s 30ms/step - loss: 0.3204 - accuracy: 0.8790 - val_loss: 0.3271 - val_accuracy: 0.8670
Epoch 3/15
215/215 [==============================] - 6s 30ms/step - loss: 0.3088 - accuracy: 0.8787 - val_loss: 0.3324 - val_accuracy: 0.8618
Epoch 4/15
215/215 [==============================] - 7s 30ms/step - loss: 0.3067 - accuracy: 0.8764 - val_loss: 0.3350 - val_accuracy: 0.8618
Epoch 5/15
215/215 [==============================] - 6s 30ms/step - loss: 0.2962 - accuracy: 0.8784 - val_loss: 0.3107 - val_accuracy: 0.8683
Epoch 6/15
215/215 [==============================] - 7s 31ms/step - loss: 0.2883 - accuracy: 0.8827 - val_loss: 0.3103 - val_accuracy: 0.8696
Epoch 7/15
215/215 [==============================] - 7s 31ms/step - loss: 0.2820 - accuracy: 0.8854 - val_loss: 0.3097 - val_accuracy: 0.870

In [52]:
y_pred =model.predict(X_test)

143/143 [==============================] - 1s 6ms/step


In [53]:
y_pred = np.where(y_pred > 0.5, 1, 0) ## AUC, ROC Curve

In [54]:
## accuracy check
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [55]:
accuracy_score(y_test, y_pred)

0.8759842519685039

In [56]:
classification_report(y_test, y_pred)

'              precision    recall  f1-score   support\n\n           0       0.96      0.81      0.88      2600\n           1       0.80      0.96      0.87      1972\n\n    accuracy                           0.88      4572\n   macro avg       0.88      0.89      0.88      4572\nweighted avg       0.89      0.88      0.88      4572\n'

In [57]:
confusion_matrix(y_test, y_pred)

array([[2119,  481],
       [  86, 1886]])